## Import and load keys

In [2]:
import os
from dotenv import load_dotenv
from mistralai import Mistral
import json
import gradio as gr
import random

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave de API de Mistral desde las variables de entorno
api_key = os.getenv('MISTRAL_API_KEY')
client = Mistral(api_key=api_key)
model = "mistral-small-latest"

## World Creation

In [3]:
system_prompt = f"""
Your job is to help create interesting fantasy worlds that \
players would love to play in.
Instructions:
- Only generate in plain text without formatting.
- Use simple clear language without being flowery.
- You must stay below 3-5 sentences for each description.
"""

world_prompt = f"""
Generate a creative description for a unique fantasy world with an
interesting concept around cities build on the backs of massive beasts.

Output content in the form:
World Name: <WORLD NAME>
World Description: <WORLD DESCRIPTION>

World Name:"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_prompt}
    ],
    temperature=0.9
)

In [4]:
world_output =output.choices[0].message.content
print(world_output)
world_output = world_output.strip()
world = {
    "name": world_output.split('\n')[0].strip()
    .replace('World Name: ', ''),
    "description": '\n'.join(world_output.split('\n')[1:])
    .replace('World Description:', '').strip()
}

World Name: The Skyborne Archipelago

World Description: In the world of The Skyborne Archipelago, cities are built on the backs of colossal, sentient beasts called Skywhales. These gentle giants glide through the endless skies, their backs hosting bustling metropolises filled with diverse cultures. The inhabitants, known as Skyborn, have adapted to life among the clouds, developing unique technologies and societies tailored to their airborne existence. Conflicts and alliances arise as different cities navigate the politics of the skies, all while the Skywhales migrate in search of the rare, mystical clouds that sustain them.


## Kingdom Creation

In [5]:
kingdom_prompt = f"""
Create 3 different kingdoms for a fantasy world.
For each kingdom generate a description based on the world it's in. \
Describe important leaders, cultures, history of the kingdom.\

Output content in the form:
Kingdom 1 Name: <KINGDOM NAME>
Kingdom 1 Description: <KINGDOM DESCRIPTION>
Kingdom 2 Name: <KINGDOM NAME>
Kingdom 2 Description: <KINGDOM DESCRIPTION>
Kingdom 3 Name: <KINGDOM NAME>
Kingdom 3 Description: <KINGDOM DESCRIPTION>

World Name: {world['name']}
World Description: {world['description']}

Kingdom 1"""


output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": kingdom_prompt}
    ],
    temperature=0.9
)

In [6]:
kingdoms = {}
kingdoms_output = output.choices[0].message.content

for output in kingdoms_output.split('\n\n'):
  kingdom_name = output.strip().split('\n')[0] \
    .split('Name: ')[1].strip()
  print(f'Created kingdom "{kingdom_name}" in {world["name"]}')
  kingdom_description = output.strip().split('\n')[1] \
    .split('Description: ')[1].strip()
  kingdom = {
      "name": kingdom_name,
      "description": kingdom_description,
      "world": world['name']
  }
  kingdoms[kingdom_name] = kingdom
world['kingdoms'] = kingdoms

print(f'\nKingdom 1 Description: \
{kingdom["description"]}')

Created kingdom "Aerium" in The Skyborne Archipelago
Created kingdom "Lumina" in The Skyborne Archipelago
Created kingdom "Veridia" in The Skyborne Archipelago

Kingdom 1 Description: Veridia is a kingdom of healers, farmers, and caretakers, led by the Verdant Matriarch. Their culture is deeply connected to nature and the well-being of the Skywhales. Veridia's history is one of nurturing and preservation, focusing on the sustainable use of the mystical clouds that sustain the Skywhales. Their city is a lush garden, teeming with life, and nestled upon the back of the nurturing Skywhale, Leafglider.


## Town Creation

In [7]:
def get_town_prompt(world, kingdom):
    return f"""
    Create 3 different towns for a fantasy kingdom and world. \
    Describe the region it's in, important places of the town, \
    and interesting history about it. \
    
    Output content in the form:
    Town 1 Name: <TOWN NAME>
    Town 1 Description: <TOWN DESCRIPTION>
    Town 2 Name: <TOWN NAME>
    Town 2 Description: <TOWN DESCRIPTION>
    Town 3 Name: <TOWN NAME>
    Town 3 Description: <TOWN DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description {kingdom['description']}
    
    Town 1 Name:"""
    
import time

import time  # Asegúrate de importar la biblioteca time

def create_towns(world, kingdom):
    print(f'\nCreating towns for kingdom: {kingdom["name"]}...')
    output = client.chat.complete(
      model=model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_town_prompt(world, kingdom)}
      ],
      temperature=0.9,
  )
    towns_output = output.choices[0].message.content
    
    towns = {}
    for output in towns_output.split('\nCreating towns for kingdom: {kingdom["name"]}...'):
        for num in range(1, 4):
            print(output.split(f"Town {num} Name: ")[1].split("\n")[0])
            print(output.split(f"Town {num} Description: ")[1].split("\n")[0])
            print("\n")
         
            town_name = output.split(f"Town {num} Name: ")[1].split("\n")[0] 
            town_description = output.split(f"Town {num} Description: ")[1].split("\n")[0]
        
            town = {
              "name": town_name,
              "description": town_description,
              "world": world['name'],
              "kingdom": kingdom['name']
            }
            towns[town_name] = town
        kingdom["towns"] = towns

In [8]:
for kingdom in kingdoms.values():
    create_towns(world, kingdom)  

first_kingdom = list(kingdoms.values())[0]
print(first_kingdom)
town = list(first_kingdom['towns'].values())[0]
print(f'\nTown 1 Description: {town["description"]}')


Creating towns for kingdom: Aerium...
Gilded Haven
Gilded Haven is a town nestled in the lush, verdant region of the Verdant Expanse, known for its abundant flora and gentle breezes. The town's most notable feature is the Golden Spire, a towering structure that serves as both a lighthouse and a beacon of hope for weary travelers. Legend has it that Gilded Haven was once a simple farming village until a mysterious golden artifact was discovered, bringing prosperity and attracting skilled artisans from across the archipelago.


Frostfall Keep
Frostfall Keep is situated in the icy, windswept peaks of the Frostwind Mountains, a region characterized by its perpetual snowfall and treacherous terrain. The town is built around the imposing Frostfall Keep, a fortress that has withstood countless sieges. Historically, Frostfall Keep was a crucial outpost in the wars against the frost giants, and its walls are adorned with the trophies of fallen foes. The town's inhabitants are renowned for thei

## Npcs Creation

In [9]:
def get_npc_prompt(world, kingdom, town): 
    return f"""
    Create 3 different characters based on the world, kingdom \
    and town they're in. Describe the character's appearance and \
    profession, as well as their deeper pains and desires. \
    
    Output content in the form:
    Character 1 Name: <CHARACTER NAME>
    Character 1 Description: <CHARACTER DESCRIPTION>
    Character 2 Name: <CHARACTER NAME>
    Character 2 Description: <CHARACTER DESCRIPTION>
    Character 3 Name: <CHARACTER NAME>
    Character 3 Description: <CHARACTER DESCRIPTION>
    
    World Name: {world['name']}
    World Description: {world['description']}
    
    Kingdom Name: {kingdom['name']}
    Kingdom Description: {kingdom['description']}
    
    Town Name: {town['name']}
    Town Description: {town['description']}
    
    Character 1 Name:"""
    

In [10]:
def create_npcs(world, kingdom, town):
    print(f'\nCreating characters for the town of: {town["name"]}...')
    output = client.chat.complete(
      model= model,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": get_npc_prompt(world, kingdom, town)}
      ],
        temperature=0.7 
    )

    npcs_output = output.choices[0].message.content
    npcs = {}
    for num in range(1, 4):
        print(npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0])
        print(npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0])
        print("\n")

        character_name = npcs_output.split(f"Character {num} Name: ")[1].split("\n")[0]
        character_description = npcs_output.split(f"Character {num} Description: ")[1].split("\n")[0]

        character = {
        "name": character_name,
        "description": character_description,
        "world": world['name'],
        "kingdom": kingdom['name'],
        "town": town['name']
        }
        npcs[character_name] = character
        town["npcs"] = npcs

    
for kingdom in kingdoms.values():
    for town in kingdom['towns'].values():
        create_npcs(world, kingdom, town)
  # For now we'll only generate npcs for one kingdom and town
    break


Creating characters for the town of: Gilded Haven...
Lyra Whisperwind
Lyra is a skilled windweaver from Aerium, with short, silver hair and piercing blue eyes that seem to hold the very essence of the sky. She wears lightweight, form-fitting armor adorned with intricate wind patterns, and her hands are always adorned with delicate, glowing wind crystals. Lyra's heart aches for the freedom of the open skies, a freedom she feels constrained by Aerium's rigid military structure. She dreams of exploring the uncharted skies and uncovering the secrets of the mystical clouds that sustain the Skywhales, hoping to bring a new era of understanding and harmony to her people.


Orion Thunderheart
Orion is a burly, bearded blacksmith from Gilded Haven, his muscular frame a testament to years of working at the forge. He has a warm, infectious laugh and eyes that sparkle with a mix of humor and sadness. Orion's life was forever changed when a tragic accident at the forge took the life of his young a

## Save World

In [11]:
def save_world(world, filename):
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        json.dump(world, f)

def load_world(filename):
    with open(filename, 'r') as f:
        return json.load(f) 

save_world(world, f'../shared_data/{world["name"]}.json')

## Generating Text box

In [12]:
demo = None
def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
                           )
    demo.launch(share=share, 
                server_name="0.0.0.0"
                )

def test_main_loop(message, history):
    return 'Entered Action: ' + message

# start_game(test_main_loop)
# start_game(main_loop)

## Generating Start

In [13]:
world_name = world["name"]
world = load_world(f'../shared_data/{world_name}.json')
# Access the world, kingdom, town and character. Here is the first of each selected.
#kingdom = random.choice(list(world['kingdoms'].keys()))
kingdom = list(world['kingdoms'].keys())[0] 
town = random.choice(list(world['kingdoms'][kingdom]['towns'].keys()))
character = random.choice(list(world['kingdoms'][kingdom]['towns'][town]['npcs'].keys()))

In [14]:
system_prompt = """You are an AI Game master. Your job is to create a 
start to an adventure based on the world, kingdom, town and character 
a player is playing as. 
Instructions:
You must only use 2-4 sentences \
Write in second person. For example: "You are Jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""
world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

model_output = client.chat.complete(
    model= model,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
    temperature=1.0
)
start = model_output.choices[0].message.content
print(start)
world['start'] = start
#save_world(world, f'../shared_data/{world_name}.json')  
save_world(world, f'../shared_data/{world_name}.json')

You are Thorne Frostbeard, a burly, bearded man from Frostfall Keep, your skin weathered by the constant cold and your eyes holding the wisdom of countless battles. As a master of ice magic, your hands are permanently frosted, a testament to your power. You stand in the heart of Frostfall Keep, the air thick with the scent of cold stone and the faint hum of magic. Around you, the keep bustles with activity, soldiers drilling and blacksmiths hammering away at their forges, while the icy winds of the Frostwind Mountains howl outside.


## Creating the Main Action Loop

In [15]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [16]:
game_state = {
    "world": world['description'],
    "kingdom": world["kingdoms"][kingdom]['description'],
    "town": world["kingdoms"][kingdom]["towns"][town]['description'],
    "character": world['kingdoms'][kingdom]['towns'][town]['npcs'][character]['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [17]:
def get_game_state(game_state=game_state, *args, **kwargs):
    world = load_world(f'../shared_data/{world_name}.json')
    kingdom = game_state["kingdom"]
    town = game_state["town"]
    character = game_state["character"]
    start = game_state['start']
    inventory = {}

    game_state = {
        "world": world,
        "kingdom": kingdom,
        "town": town,
        "character": character,
        "start": start,
        "inventory": game_state.get('inventory', {})
    }
    return game_state

## Launch and Share!

In [18]:
# start_game(main_loop, True)

In [19]:
# from helper import get_game_state

# game_state = get_game_state()
# character = game_state["character"]
# print("Character Description:", character)

## Define Inventory Detector

In [20]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""

def detect_inventory_changes(game_state, output):
    inventory = game_state.get('inventory', {})
    
    # Prompt más estricto para Mistral
    system_prompt = """You are an inventory tracker. Respond ONLY with valid JSON in this exact format:
{
    "itemUpdates": [
        {"name": "item_name", "change_amount": 1}
    ]
}"""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {inventory}"},
        {"role": "user", "content": f"Action: {output}"},
        {"role": "user", "content": "List item changes in the specified JSON format."}
    ]
    
    # Llamada a Mistral
    response = client.chat.complete(
        model=model,
        temperature=0.0,
        messages=messages
    ).choices[0].message.content
    try:
        cleaned = response.strip()
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:-3].strip()
        elif cleaned.startswith('```'):
            cleaned = cleaned[3:-3].strip()
        
        return json.loads(cleaned).get('itemUpdates', [])
    
    except Exception as e:
        print(f"Error parsing inventory: {str(e)}")
        return []

In [21]:
def detect_inventory_changes(game_state, output):
    inventory = game_state.get('inventory', {})
    
    # Prompt actualizado y más explícito para forzar siempre una respuesta en JSON.
    system_prompt = (
        "You are an inventory tracker. You are given the player's current inventory and an action narrative. "
        "The narrative may include summaries (e.g., 'You have 4 sketches, 4 diagrams, ...') which should be ignored. "
        "Only report inventory changes when the narrative explicitly indicates that the player picks up, acquires, loses, or drops an item. "
        "Do not infer changes from inventory summaries. "
        "Your answer must always include valid JSON with the key 'itemUpdates'. "
        "If there are no explicit changes, return exactly: {\"itemUpdates\": []}. "
        "Ensure your response is not empty."
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {inventory}"},
        {"role": "user", "content": f"Action Narrative: {output}"},
        {"role": "user", "content": "List explicit inventory changes in the specified JSON format."}
    ]
    
    response = client.chat.complete(
        model=model,
        temperature=0.0,
        messages=messages
    ).choices[0].message.content
    
    try:
        cleaned = response.strip()
        # Si la respuesta es vacía, se retorna sin actualizaciones
        if not cleaned:
            return []
        # Quitar delimitadores de código si están presentes
        if cleaned.startswith('```json'):
            cleaned = cleaned[7:-3].strip()
        elif cleaned.startswith('```'):
            cleaned = cleaned[3:-3].strip()
        
        return json.loads(cleaned).get('itemUpdates', [])
    
    except Exception as e:
        print(f"Error parsing inventory: {str(e)}")
        return []

In [22]:
def update_inventory(inventory, item_updates):
    update_msg = ''
    
    for update in item_updates:
        name = update['name']
        change_amount = update['change_amount']
        
        if change_amount > 0:
            if name not in inventory:
                inventory[name] = change_amount
            else:
                inventory[name] += change_amount
            update_msg += f'\nInventory: {name} +{change_amount}'
        elif name in inventory and change_amount < 0:
            inventory[name] += change_amount
            update_msg += f'\nInventory: {name} {change_amount}'
            
        if name in inventory and inventory[name] < 0:
            del inventory[name]
            
    return update_msg

## Story with inventory

In [23]:
def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']
        
    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...) \
Don't let the player use items they don't have in their inventory.
"""

    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}
Inventory: {json.dumps(game_state['inventory'])}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]

    for user_msg, bot_resp in history:
        messages.append({"role": "assistant", "content": bot_resp})
        messages.append({"role": "user", "content": user_msg})
           
    messages.append({"role": "user", "content": message})
    if not message or not history:
        return "Estado del juego no válido"
    model_output = client.chat.complete(
        model=model,
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [24]:
game_state = get_game_state(inventory={
    "cloth pants": 1,
    "cloth shirt": 1,
    "goggles": 1,
    "leather bound journal": 1,
    "gold": 5
})

## Start the game (inventory 1)

In [25]:
def main_loop(message, history):
    try:
        # Validación básica del input
        if not message.strip():
            return "Por favor escribe una acción válida"
        
        # 1. Ejecutar acción principal
        output = str(run_action(message, history, game_state))  # Aseguramos string
        
        # 2. Validar seguridad
        #if not is_safe(output):
        #    return "⚠️ Esa acción no está permitida en este momento"
        
        # 3. Procesar inventario (con reintentos)
        max_retries = 3
        inventory_msg = ""
        
        for attempt in range(max_retries):
            try:
                item_updates = detect_inventory_changes(game_state, output)
                if item_updates:
                    inventory_msg = update_inventory(game_state['inventory'], item_updates)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"⚠️ Fallo al actualizar inventario: {str(e)}")
                time.sleep(1)  # Espera entre reintentos
        
        # Construir respuesta final
        response = output
        if inventory_msg:
            response += f"\n\n🔹 {inventory_msg}"
            
        return response
        
    except Exception as e:
        # Manejo detallado del error
        error_type = type(e).__name__
        
        if str(e) == "0":
            print("🔴 Error crítico: Posible fallo en la API o respuesta vacía")
            return "El sistema no pudo procesar tu acción. Intenta algo diferente."
        else:
            print(f"🔴 Error crítico ({error_type}): {str(e)}")
            return f"Error: {error_type}. Por favor intenta nuevamente."

start_game(main_loop)

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12148\730323021.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
c:\Users\USUARIO\AppData\Local\pypoetry\Cache\virtualenvs\llm-game-xAnBDEWD-py3.13\Lib\site-packages\gradio\chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


## Game with inventory type 2

In [28]:
def main_loop(message, history):
    try:
        # Validación básica del input
        if not message.strip():
            return "Por favor escribe una acción válida"
        
        # 1. Ejecutar acción principal
        output = str(run_action(message, history, game_state))  # Aseguramos string
        
        # 2. Validar seguridad (comentado)
        # if not is_safe(output):
        #     return "⚠️ Esa acción no está permitida en este momento"
        
        # 3. Procesar inventario (con reintentos)
        max_retries = 3
        inventory_msg = ""
        
        for attempt in range(max_retries):
            try:
                item_updates = detect_inventory_changes(game_state, output)
                # Filtrar solo actualizaciones válidas (que tengan las claves 'name' y 'change_amount')
                valid_updates = [
                    update for update in item_updates
                    if isinstance(update, dict) and 'name' in update and 'change_amount' in update
                ]
                if valid_updates:
                    inventory_msg = update_inventory(game_state['inventory'], valid_updates)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"⚠️ Fallo al actualizar inventario: {str(e)}")
                time.sleep(1)  # Espera entre reintentos
        
        # Construir respuesta final
        response = output
        if inventory_msg:
            response += f"\n\n🔹 {inventory_msg}"
            
        return response
        
    except Exception as e:
        # Manejo detallado del error
        error_type = type(e).__name__
        if str(e) == "0":
            print("🔴 Error crítico: Posible fallo en la API o respuesta vacía")
            return "El sistema no pudo procesar tu acción. Intenta algo diferente."
        else:
            print(f"🔴 Error crítico ({error_type}): {str(e)}")
            return f"Error: {error_type}. Por favor intenta nuevamente."

start_game(main_loop)

Closing server running on port: 7860


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_12148\730323021.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
c:\Users\USUARIO\AppData\Local\pypoetry\Cache\virtualenvs\llm-game-xAnBDEWD-py3.13\Lib\site-packages\gradio\chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


## Api Test

In [27]:
def check_api_status():
    try:
        test_response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": "Responde 'OK'"}],
            temperature=0.0
        )
        return True if test_response.choices[0].message.content else False
    except:
        return False

if not check_api_status():
    print("🔴 La API de Mistral no está respondiendo")
    # Puedes agregar un reintento o salir

check_api_status()

True